# Data Processing and Model Training Notebook

In [20]:
# import packages
import os
import cv2
import numpy as np
from IPython.display import Video, display

from utils.pretrained_deployment import download_images, download_images2, download_images_with_resize, download_images_full_size
from utils.display import *
from utils.make_dataset_nyc_landmarks import make_nyc_dataset

from bing_image_downloader import downloader

device = 'cuda'

# reload modules every 2 seconds to see changes in notebook
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Download the Images

In [32]:
def download_images(query, num_images, download_path):
    downloader.download(query, limit=num_images, output_dir=download_path, adult_filter_off=True, force_replace=False)


# classes = ["empire_state_building", 
# "1_world_trade_center", 
# "chrysler_building", 
# "432_park_ave", 
# "statue_of_liberty",
# "brooklyn_bridge",
# "vessel_nyc",
# "united_nations_building",
# "grand_central_terminal",
# "flatiron"]

queries = ["empire_state_building_from_the_ground", 
"1_world_trade_center", 
"432_park_ave", 
"united_nations_building",
"flatiron"]

classes = {
    0: "EmpireState",
    1: "WTC",
    2: "432ParkAve",
    3: "UNBuilding",
    4: "Flatiron"
}
#Make sure the yaml file matches this ^^



# query = 'empire_state_building_from_the_ground'
num_images = 5  # Number of images to download
download_path = "./downloaded_images"  # Target folder path
for query in queries:
    # download_images_with_resize(query, num_images, target_size=target_size)
    # download_images_full_size(query, num_images, download_folder)
    download_images(query, num_images, download_path)

[%] Downloading Images to /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/downloaded_images/empire_state_building_from_the_ground


[!!]Indexing page: 1

[%] Indexed 5 Images on Page 1.


[%] Downloading Image #1 from https://c1.staticflickr.com/5/4092/5042429689_582d127a7f_z.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://i.pinimg.com/originals/96/9b/bd/969bbdc56048c56de553ac908c20ac16.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://blogs.voanews.com/tedlandphairsamerica/files/2012/05/02-from-ground-level-cmh1.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://geographic.media/wp-content/uploads/us0012-1.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://www.findingtheuniverse.com/wp-content/uploads/2020/07/Empire-State-Building-from-ground-level_by_Laurence-Norah-2.jpg
[%] File Downloaded !



[%] Done. Downloaded 5 images.


# Add annotations using rectlabel

Use the available software to create the yolo annotations for each image. This will take some time so go ahead, put some podcasts and get cracking. The .txt files need to be saved in the same directory as the original images. Make sure that the labels are following the same classes dictionary that we have here or things will not go well. 

# Place into valid and training folders, prepare yolo datasets

In [33]:
#########################################################
# ONLY RUN IF YOU WANT DELETE CURRENT DATA AND REMAKE SET
#########################################################

fraction_valid = 0.25
train_path = "./datasets/nyc_landmarks/train"
valid_path = "./datasets/nyc_landmarks/valid"

make_nyc_dataset(
    download_path,
    train_path = train_path,
    val_path = valid_path,
    label_dictionary = classes,
    fraction_valid = fraction_valid
)

Processing folder: 432_park_ave
Raw Name: Image_3
Combo Name: 432_park_ave_Image_3
Raw Name: Image_2
Combo Name: 432_park_ave_Image_2
Raw Name: Image_1
Combo Name: 432_park_ave_Image_1
Raw Name: Image_5
Combo Name: 432_park_ave_Image_5
Raw Name: Image_4
Combo Name: 432_park_ave_Image_4
Processing folder: united_nations_building
Raw Name: Image_3
Combo Name: united_nations_building_Image_3
Raw Name: Image_2
Combo Name: united_nations_building_Image_2
Raw Name: Image_1
Combo Name: united_nations_building_Image_1
Raw Name: Image_5
Combo Name: united_nations_building_Image_5
Raw Name: Image_4
Combo Name: united_nations_building_Image_4
Processing folder: 1_world_trade_center
Raw Name: Image_3
Combo Name: 1_world_trade_center_Image_3
Raw Name: Image_2
Combo Name: 1_world_trade_center_Image_2
Raw Name: Image_1
Combo Name: 1_world_trade_center_Image_1
Raw Name: Image_5
Combo Name: 1_world_trade_center_Image_5
Raw Name: Image_4
Combo Name: 1_world_trade_center_Image_4
Processing folder: empire

## Verify Dataset

# Train Yolov9

In [35]:
from ultralytics import YOLO

# Build a YOLOv9c model from pretrained weight
model = YOLO('yolov9c.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)

100%|██████████| 49.4M/49.4M [00:05<00:00, 9.79MB/s]


YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs
Ultralytics YOLOv8.1.29 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=dataset.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False

train: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<00:00, 129.35it/s]

train: New cache created: /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train.cache



val: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<00:00, 4700.03it/s]

...

Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.506       5.19      1.773         12        640: 100%|██████████| 2/2 [01:04<00:00, 32.27s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.10s/it]


                   all          5          5   0.000362        0.2    0.00474    0.00237

1 epochs completed in 0.020 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 51.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M2)
YOLOv9c summary (fused): 384 layers, 25323103 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.94s/it]


                   all          5          5   0.000361        0.2    0.00498    0.00249
           EmpireState          5          1          0          0          0          0
                   WTC          5          1          0          0          0          0
            432ParkAve          5          1          0          0          0          0
            UNBuilding          5          1    0.00181          1     0.0249     0.0124
              Flatiron          5          1          0          0          0          0
Speed: 1.1ms preprocess, 556.1ms inference, 0.0ms loss, 15.2ms postprocess per image
Results saved to runs/detect/train


# Evaluation

In [44]:
from PIL import Image
import matplotlib.pyplot as plt

# Predict bounding boxes on validation data
results = model.predict(valid_path)


# for result in results:
#     print(result)




image 1/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/1_world_trade_center_Image_3.jpg: 640x448 (no detections), 340.7ms
image 2/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/432_park_ave_Image_3.jpg: 640x448 (no detections), 329.4ms
image 3/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/empire_state_building_from_the_ground_Image_3.jpg: 640x448 (no detections), 329.0ms
image 4/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/flatiron_Image_3.jpg: 640x448 (no detections), 314.0ms
image 5/5 /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid/united_nations_building_Image_3.jpg: 480x640 (no 

# Train RT-DETR

In [45]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='dataset.yaml', epochs=1, imgsz=640)

100%|██████████| 63.4M/63.4M [00:08<00:00, 8.11MB/s]


rt-detr-l summary: 673 layers, 32970476 parameters, 0 gradients, 108.3 GFLOPs
Ultralytics YOLOv8.1.29 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=dataset.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

train: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/train.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
val: Scanning /Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-final-project-jws2215/datasets/nyc_landmarks/valid.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 143 weight(decay=0.0), 206 weight(decay=0.0005), 226 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train2
Starting training for 1 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


        1/1         0G      1.016      40.07      1.353         12        640: 100%|██████████| 2/2 [01:16<00:00, 38.36s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.73s/it]

                   all          5          5          0          0          0          0



1 epochs completed in 0.023 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 66.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 66.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.11.7 torch-2.2.1 CPU (Apple M2)
rt-detr-l summary: 498 layers, 31994015 parameters, 0 gradients, 103.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.55s/it]

                   all          5          5          0          0          0          0


Speed: 0.9ms preprocess, 494.1ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train2
